# Assignment 1

Name: Pola Gnana Shekar

Roll No: 21CS10052

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../../dataset/decision-tree.csv')
print(df.shape)

(768, 9)


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load the dataset
dataset_path = '../../dataset/decision-tree.csv'
data = pd.read_csv(dataset_path)

# Create an imputer instance
imputer = SimpleImputer(strategy='mean')

# Impute missing values in the entire dataset
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

def normalize_data(data):
    # Exclude the "Outcome" column from normalization
    features = data.drop(columns=['Outcome'])
    
    # Calculate the minimum and maximum values for each feature
    min_vals = features.min()
    max_vals = features.max()
    
    # Perform normalization for each feature
    normalized_features = (features - min_vals) / (max_vals - min_vals)
    
    # Combine the normalized features with the "Outcome" column
    normalized_data = pd.concat([normalized_features, data['Outcome']], axis=1)
    
    return normalized_data

# Normalize the data before splitting
data = normalize_data(data)

# Split the data into training and testing sets using scikit-learn
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Implement the ID3 Decision Tree algorithm
class TreeNode:
    def __init__(self, attribute=None, label=None):
        self.attribute = attribute
        self.label = label
        self.children = {}

def entropy(target):
    # Calculate the entropy of a target variable
    unique_labels, counts = np.unique(target, return_counts=True)
    probabilities = counts / len(target)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def information_gain(data, target, attribute):
    total_entropy = entropy(target)
    values, counts = np.unique(data[attribute], return_counts=True)
    probabilities = counts / len(data)

    weighted_entropy = np.sum(probabilities * [entropy(target[data[attribute] == value]) for value in values])

    information_gain = total_entropy - weighted_entropy
    return information_gain

def id3_decision_tree(data, attributes, target_attribute, depth=0, max_depth=11):
    unique_classes = data[target_attribute].unique()
    
    # Create a tree node
    if len(unique_classes) == 1:
        return TreeNode(label=unique_classes[0])

    if len(attributes) == 0 or depth >= max_depth:
        majority_class = data[target_attribute].mode()[0]
        return TreeNode(label=majority_class)

    # Find the best attribute to split on
    best_attribute = select_best_attribute(data, attributes, target_attribute)

    root = TreeNode(attribute=best_attribute)

    # Split the data on the best attribute
    attribute_values = data[best_attribute].unique()
    for value in attribute_values:
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            majority_class = data[target_attribute].mode()[0]
            root.children[value] = TreeNode(label=majority_class)
        else:
            root.children[value] = id3_decision_tree(subset, attributes.drop(best_attribute), target_attribute, depth + 1, max_depth)

    return root

def select_best_attribute(data, attributes, target_attribute):
    best_attribute = None
    best_information_gain = -1

    for attribute in attributes:
        information_gain_value = information_gain(data, data[target_attribute], attribute)
        if information_gain_value > best_information_gain:
            best_information_gain = information_gain_value
            best_attribute = attribute

    return best_attribute

def reduced_error_pruning(node, data, target):
    if not node.children:
        return

    # Prune children nodes
    for value, child_node in node.children.items():
        reduced_error_pruning(child_node, data, target)

    # Calculate error without pruning (using the original unpruned tree)
    predictions_before_pruning = [predict(node, data_point) for data_point in data.to_numpy()]
    error_before_pruning = np.sum(np.array(predictions_before_pruning) != np.array(target))

    # Prune the current node
    old_children = node.children.copy()
    node.children = {}

    # Calculate error after pruning (using the pruned tree)
    predictions_after_pruning = [predict(node, data_point) for data_point in data.to_numpy()]
    error_after_pruning = np.sum(np.array(predictions_after_pruning) != np.array(target))

    # If pruning improves accuracy or doesn't change it, keep the pruning; otherwise, revert to old children
    if error_after_pruning <= error_before_pruning:
        print(f"Pruned node: {node.attribute}, Depth: {node.depth}, Error before pruning: {error_before_pruning}, Error after pruning: {error_after_pruning}")
    else:
        node.children = old_children

# Define a function to predict the outcome for a single data point
def predict(node, data_point):
    if not node.children:
        return node.label  # Return the label for leaf nodes

    if node.attribute is None:
        # This is a leaf node with a label
        return node.label

    value = data_point[node.attribute]

    if value not in node.children:
        print(f"Warning: Value {value} not found in node {node.attribute}. Using majority class.")
        return np.argmax(np.bincount(node.label))

    child_node = node.children[value]
    return predict(child_node, data_point)

# Define functions to calculate evaluation metrics
def macro_precision(predictions, true_labels):
    labels = np.unique(true_labels)
    precision = []
    for label in labels:
        true_positive = np.sum((predictions == label) & (true_labels == label))
        false_positive = np.sum((predictions == label) & (true_labels != label))
        if true_positive + false_positive == 0:
            precision.append(0)
        else:
            precision.append(true_positive / (true_positive + false_positive))
    return np.mean(precision)

def macro_recall(predictions, true_labels):
    labels = np.unique(true_labels)
    recall = []
    for label in labels:
        true_positive = np.sum((predictions == label) & (true_labels == label))
        false_negative = np.sum((predictions != label) & (true_labels == label))
        if true_positive + false_negative == 0:
            recall.append(0)
        else:
            recall.append(true_positive / (true_positive + false_negative))
    return np.mean(recall)

# Define a function to print the pruned decision tree structure
def print_pruned_tree(node, depth):
    if node is None:
        return
    indent = "  " * depth
    if node.attribute is not None:
        print(f"{indent}Attribute: {node.attribute}")
    if node.value is not None:
        print(f"{indent}Value: {node.value}")
    
    if node.children:
        for value, child_node in node.children.items():
            # Check if the child node has a valid attribute
            if child_node.attribute is not None:
                print(f"{indent}  If {child_node.attribute} == {value}:")
                print_pruned_tree(child_node, depth + 1)
            else:
                print(f"{indent}  Predicted Outcome: {predict(child_node, pd.Series(node.data))}")

# Create a variable containing all columns other than "Outcome"
attr = train_data.drop(columns=['Outcome'])

# Build the ID3 decision tree
tree = id3_decision_tree(train_data, attr.columns, 'Outcome', 0, 11)

# Perform reduced error pruning on the tree
reduced_error_pruning(tree, train_data, train_data['Outcome'])

# Calculate train and test accuracies at different depths
depths = list(range(1, 11))
train_accuracies = []  # List to store training accuracies
test_accuracies = []   # List to store test accuracies

for depth in depths:
    pruned_tree = id3_decision_tree(train_data, train_data.drop(columns=['Outcome']), 'Outcome', depth, max_depth=depth)
    
    # Calculate training accuracy
    train_predictions = [predict(pruned_tree, train_data.iloc[i]) for i in range(len(train_data))]
    train_accuracy = np.sum(train_predictions == train_data['Outcome']) / len(train_data)
    train_accuracies.append(train_accuracy)
    
    # Calculate test accuracy
    test_predictions = [predict(pruned_tree, test_data.iloc[i]) for i in range(len(test_data))]
    test_accuracy = np.sum(test_predictions == test_data['Outcome']) / len(test_data)
    test_accuracies.append(test_accuracy)
    
     # Print the accuracies
    print(f"Depth {depth}: Training Accuracy = {train_accuracy:.2f}, Test Accuracy = {test_accuracy:.2f}")

# Plot the variation in training and test accuracy with varying depths
plt.plot(depths, train_accuracies, marker='o', label='Training Accuracy')
plt.plot(depths, test_accuracies, marker='o', label='Test Accuracy')
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy vs. Tree Depth')
plt.legend()  # Add legend to differentiate between training and test accuracy
plt.grid(True)
plt.show()

# Print the pruned decision tree structure after pruning
print("Pruned Decision Tree After Pruning:")
print_pruned_tree(tree, 0)

# Make predictions using the pruned tree on the test data
predictions = [predict(tree, test_data.iloc[i]) for i in range(len(test_data))]

# Calculate evaluation metrics
accuracy = np.sum(predictions == test_data['Outcome']) / len(test_data)
precision = macro_precision(predictions, test_data['Outcome'])
recall = macro_recall(predictions, test_data['Outcome'])

print(f"Test Accuracy: {accuracy}")
print(f"Macro Precision: {precision}")
print(f"Macro Recall: {recall}")

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices